Imports and settings

In [ ]:
# auto-reload modules
%load_ext autoreload
%autoreload 2
# display figures in svg format
%config InlineBackend.figure_formats = ['svg']

# imports
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sys.path.append("../")

from per_analysis import config
from per_analysis.figures import lineplot, rasterplot, savefig
from per_analysis.io import read_data
from per_analysis.stats import estimate_mode_1d

# set maximum dimensions for displaying dataframes
pd.set_option("display.max_columns", 27)

# load custom matplotlib style sheet
plt.style.use("../styles/custom.mplstyle")

Read data

In [ ]:
df = pd.read_csv("../data/WT_50%.csv")

Show the first few rows of the data

In [ ]:
df.head(5)

List flies with incorrect number of frames

In [ ]:
frame_count = (
    df.groupby(["fly", "trial"])
    .count()
    .unstack("trial")["haustellum_angle"]
)

frame_count[(frame_count != config.N_FRAMES_PER_TRIAL).any(axis=1)]

Remove flies with errors

In [ ]:
discard = (
    "20190904-1",  # trial 10 only has 36 frames
    "20190904-10",  # inaccurate tracking (e.g., 20190904-10-007 to 20190904-10-09)
    "20190905-5",  # fly fell off in trial 6
)

df = df[~df.fly.isin(discard)]

Estimate angle baselines and subtract baselines from the angles for comparison between flies

In [ ]:
baseline = {
    angle: {
        fly: estimate_mode_1d(df_fly[angle], dx=1)
        for fly, df_fly in df.groupby("fly")
    }
    for angle in ("rostrum_angle", "haustellum_angle")
}

for angle in ("rostrum_angle", "haustellum_angle"):
    for fly in df.fly.unique():
        df.loc[df.fly == fly, angle] -= baseline[angle][fly]

Add additional columns for plotting

In [ ]:
df["time"] = df.frame / config.FRAME_RATE
df["trial_name"] = (df.trial % 3 + 1).astype(str)
# display first few rows of the dataframe

Settings for figures. Figures will be save in `../figures/`.

In [ ]:
fig_dir = Path("../figures/")
trial_palette = sns.palettes.husl_palette(3, s=1, l=0.6)
angle_labels = dict(
    rostrum_angle="Rostrum angle (°)", haustellum_angle="Haustellum angle (°)"
)
angle_cmaps = dict(rostrum_angle="Oranges", haustellum_angle="Blues")
angle_colors = dict(rostrum_angle="C1", haustellum_angle="C0")

Angle to plot (`"rostrum_angle"` or `"haustellum_angle"`)

In [ ]:
angle = "rostrum_angle"

Plot individual flies for data verification. Results are saved to the figure directory.

In [ ]:
for fly, df_fly in df.groupby("fly"):
    df_fly = df_fly.copy()
    df_fly[angle] += baseline[angle][fly]
    g = lineplot(
        df_fly,
        x="time",
        y=angle,
        xlabel="Time (s)",
        ylabel=angle_labels[angle],
        hlines=baseline[angle][fly],
        vlines=config.ODOR_INTERVAL,
        spines=True,
        lineplot_kws={
            "lw": 0.5,
            "hue": "trial_name",
            "estimator": None,
            "palette": trial_palette,
        },
        wspace=0.1,
        hspace=0.3,
    )
    g.add_legend(
        title="Trial",
        handlelength=1,
        borderaxespad=1,
        borderpad=0,
        loc="center left",
        bbox_to_anchor=(1, 0.5),
        bbox_transform=g.axes[5].transAxes,
    )
    savefig(fig_dir / "individual_fly" / angle / f"{fly}.svg")
    plt.close()

Angle averaged over flies
- Shaded areas are 95% confidence intervals
- Horizontal dotted lines indicate the baseline (angle at resting state)
- Vertical dotted lines indicate the interval within which the odor was presented

In [ ]:
fig = lineplot(
    df,
    x="time",
    y=angle,
    xlabel="Time (s)",
    ylabel=angle_labels[angle],
    spines=False,
    hlines=(0,),
    vlines=config.ODOR_INTERVAL,
    lineplot_kws={
        "lw": 1,
        "ci": 95,
        "err_kws": {"lw": 0},
        "color": angle_colors[angle],
    },
    wspace=0.1,
    hspace=0.3,
    y_scale_bar_length=30,
)
savefig(fig_dir / f"{angle}_mean_ci_95.svg")

Raster plot
- Each row represents 1 fly
- For each fly, 3 trials of the same odor are concatenated (15 s x 3 = 45 s)
- Baseline (angle at resting state) was subtracted for inter-fly comparison
- Black triangles indicate the interval within which the odor was presented

In [ ]:
fig = rasterplot(
    df,
    x="time",
    y=angle,
    ylabel="Fly",
    vlines=config.ODOR_INTERVALS_3_TRIALS,
    vline_kws=dict(
        marker=6,
        markersize=3,
        markeredgewidth=0,
        markevery=2,
        clip_on=False,
        mfc="k",
        lw=0,
    ),
    cb_label=angle_labels[angle],
    cmap=angle_cmaps[angle],
    time_scale_bar=5,
    cb_height=1.5,
)
savefig(fig_dir / f"{angle}_raster.svg")

PER duration calculation. PER is defined as when $\theta_{haustellum} \geq 90^\circ$.

In [ ]:
haustellum_angle_threshold = 90
df["time_bin"] = df.time.astype(int) + 0.5
df["haustellum_angle_exceeds_threshold"] = (
    df.haustellum_angle >= haustellum_angle_threshold
)
df_per_duration = (
    df.groupby(["fly", "odor", "trial_name", "time_bin"])
    .mean()
    .haustellum_angle_exceeds_threshold.to_frame("per_duration")
    .reset_index()
)
df_per_duration

PER duration
- Error bars are 95% confidence intervals
- Shaded area indicates the odor presentation period

In [ ]:
lineplot(
    df_per_duration,
    x="time_bin",
    y="per_duration",
    xlabel="Time (s)",
    ylabel="PER duration (s)",
    vspans=((2, 4),),
    vspan_kws=dict(lw=0, fc="k", alpha=0.1),
    xmargin=None,
    lineplot_kws=dict(
        ci=95,
        color="k",
        err_style="bars",
        lw=0.5,
        err_kws=dict(elinewidth=0.5, capsize=1, capthick=0.5),
        marker="D",
        markersize=2,
        markeredgewidth=0,
    ),
)
savefig(fig_dir / f"per_duration.svg")

Sum PER duration from 2 to 7 s to get the integrated PER duration. The PER probability is 1.0 for trials whose
integrated PER duration is greater than 0. Otherwise, the PER probability is 0.

In [ ]:
df_integrated_per_duration = df_per_duration[(df_per_duration.time_bin >= 2) & (df_per_duration.time_bin < 7)].groupby(["fly", "odor", "trial_name"]).sum()[["per_duration"]].rename({"per_duration": "integrated_per_duration"})
df_integrated_per_duration["per_probability"] = (df_integrated_per_duration.per_duration != 0).astype(float)
df_integrated_per_duration

Plot PER probability. Error bars are 95% confidence interval.

In [ ]:
fig, ax = plt.subplots(figsize=(4, 2))
sns.barplot(
    data=df_integrated_per_duration.reset_index(),
    x="odor",
    y="per_probability",
    order=config.ODOR_LINEAR_ORDER,
    color="k",
    errcolor="k",
    errwidth=1,
    capsize=.2,
    ci=95,
    n_boot=1000,
    ax=ax,
)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.set_ylabel("PER probability")
ax.set_xlabel(None)
savefig(fig_dir / f"per_probability.svg")